# Demo of MRIO and EE classes

This notebook demonstrates the `MRIO` and `EE` classes that make IO calculations easier in Python. 

In [1]:
import duckdb
from mrio import MRIO, EE

Before running this notebook, save the EE-MRIO Excel files in `data/raw` and run `1-preprocess-eemrios.py`. This compiles all years into two files: `mrio.parquet` and `ee.parquet`. These store the MRIOs in a lightweight, machine-readable format. 

We can view snippets below:

In [2]:
input_mrio = '../data/mrio.parquet'
input_ee = '../data/ee.parquet'

In [3]:
duckdb.sql(f"SELECT * FROM '{input_mrio}' LIMIT 5").df()

,t,si,AUS_c1,AUS_c2,AUS_c3,AUS_c4,AUS_c5,AUS_c6,AUS_c7,AUS_c8,...,NZL_F2,NZL_F3,NZL_F4,NZL_F5,RoW_F1,RoW_F2,RoW_F3,RoW_F4,RoW_F5,ToT
0,2017,AUS_c1,12199.559183,214.280421,22912.845631,634.534126,54.032134,1754.892087,19.475614,19.776085,...,0.0,0.306421,4.398006,-2.632899,755.319306,0.000078,5.577177,9.456021,0.021587,76590.551093
1,2017,AUS_c2,250.316692,16671.934616,481.699583,9.676875,1.191425,44.117673,291.621069,1286.044207,...,0.0,0.000000,11.431150,21.523687,6.704805,0.001411,0.686634,28.234911,0.003934,219904.607558
2,2017,AUS_c3,1088.256793,320.590655,6280.283519,5.488313,162.329879,16.741351,36.602804,42.586943,...,0.0,0.000000,0.000000,15.801672,764.151993,0.019817,9.609331,0.000134,0.132746,68876.130878
3,2017,AUS_c4,18.164412,36.927218,21.299460,25.558736,1.896831,6.166664,19.265706,1.765148,...,0.0,0.000000,1.204813,-0.370723,155.353612,0.004976,1.319984,0.002417,-0.012125,3577.508549
4,2017,AUS_c5,3.681203,6.056107,7.035795,24.693916,15.141007,0.715095,12.437081,0.353283,...,0.0,0.000000,0.000000,-0.542867,23.747787,0.000040,0.246063,0.057996,0.829850,830.406474


In [4]:
duckdb.sql(f"SELECT * FROM '{input_ee}' LIMIT 5").df()

,t,activity,gas,sector,AUS_c1,AUS_c2,AUS_c3,AUS_c4,AUS_c5,AUS_c6,...,NZL_F1,NZL_F2,NZL_F3,NZL_F4,NZL_F5,RoW_F1,RoW_F2,RoW_F3,RoW_F4,RoW_F5
0,2017,Main Activity Electricity and Heat Production,CO2,Energy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2017,Petroleum Refining - Manufacture of Solid Fuel...,CO2,Energy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,2017,Manufacturing Industries and Construction,CO2,Energy,193.561566,8700.295764,5408.680335,134.406273,18.756567,1832.034981,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,2017,Civil Aviation,CO2,Energy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,2017,Road Transportation no resuspension,CO2,Energy,5336.520981,11666.858886,132.526289,54.347547,20.084639,35.335576,...,5704.711278,0.0,0.0,0.0,0.0,293028.417851,0.0,0.0,0.0,0.0


The `MRIO` and `EE` classes extract all relevant matrices and vectors from the MRIO to facilitate matrix operations. The `MRIO` class accepts the arguments `file_path` and `year` while the `EE` class accepts the arguments `file_path`, `year`, and `by`. The argument `by` can have value `None` (default), `gas`, or `sector`. This determines how the EE matrix is aggregated.

To demonstrate:

In [5]:
year = 2021
mrio = MRIO(f'{input_mrio}', year)
ee = EE(f'{input_ee}', year, by='gas')

In [6]:
x = mrio.x
x.shape

(2555,)

In [7]:
E = ee.E
E.shape

(4, 2555)

The matrix `E` is a `4x(G*N)` matrix where the rows are the four types of gases in the EE-MRIO (by virtue of having set `by = 'gas'` in the `EE` instantiation). The gases can be called using the `rows` property:

In [8]:
ee.rows

array([['CH4'],
       ['CO2'],
       ['GWP_100_AR5_F-gases'],
       ['N2O']], dtype=object)

Computing the `EBY` matrix requires dividing the `E` matrix by the output vector `x`. This is done using the built-in `diag()` method:

In [9]:
e = E @ (1/x).diag()
e.shape

(4, 2555)

## Demo with dummy MRIO

The best way to demonstrate the uses of the `MRIO` class is by example. The following dummy MRIO will be used. This has three countries, two sectors, one final demand category, and one value added category.

In [10]:
duckdb.sql("SELECT * FROM '../data/dummy_mrio.parquet'").df()

,t,si,C1,C2,J1,J2,U1,U2,Cf,Jf,Uf,ToT
0,2020,C1,0,0,0,2.0,0,0,1.0,0.0,0.0,3.0
1,2020,C2,0,0,0,0.0,0,0,1.0,0.0,0.0,1.0
2,2020,J1,0,0,0,0.0,0,0,0.0,2.0,0.0,2.0
3,2020,J2,0,0,0,0.0,0,0,0.0,1.0,2.5,3.5
4,2020,U1,0,0,0,0.0,0,0,0.0,0.0,2.0,2.0
5,2020,U2,0,0,0,0.0,0,0,0.0,0.0,1.0,1.0
6,2020,VA,3,1,2,1.5,2,1,NaN,NaN,NaN,NaN
7,2020,ToT,3,1,2,3.5,2,1,NaN,NaN,NaN,NaN


This is loaded as an instance of the `MRIO` class.

In [11]:
mrio = MRIO('../data/dummy_mrio.parquet', year=2020, N=2, f=1, vc=1, full=True)

The key vectors and matrices of the MRIO are accessible via properties. It will be useful to save them as variables.

In [12]:
x = mrio.x
Z = mrio.Z
va = mrio.va
Y = mrio.Y
A = mrio.A
B = mrio.B

The values of each can be accessed with the `data` property. To illustrate with the intermediates matrix Z:

In [13]:
Z.data

array([[0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

The following methods are built into the `MRIO` class. These help rearrange and manipulate the MRIO elements for IO computations, particularly exports decomposition. Some of these are discussed in more detail below.

- `diag()`: wrapper for `np.diag()`
- `invert()`: wrapper for `np.linalg.inv()`
- `kron()`: wrapper for `np.kron()`
- `t()`: wrapper for `np.transpose()`
- `col_sum(chunk=None)` and `row_sum(chunk=None)`
- `subset(row=None, col=None)`
- `asvector()`
- `zeroout(row=None, col=None, inverse=False)`
- `get_fatdiag()`
- `diagvec()`
- `diagmat(offd=False)`
- `diagrow()`
- `diagstack(by='row')`

### `subset()`

Subsets a vector or matrix by the country index. For example, to get the portion of the Z matrix corresponding to country 1 exports of intermediates to countries 2 and 3, simply write:

In [14]:
print('Z =\n', Z.data, '\n\nZ.subset(row=1, col=[2, 3]) = \n', Z.subset(row=1, col=[2, 3]).data)

Z =
 [[0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]] 

Z.subset(row=1, col=[2, 3]) = 
 [[0. 2. 0. 0.]
 [0. 0. 0. 0.]]


If either `row` or `col` is not specified, then the all countries are returned. If a negative index is supplied, then everything but that country is returned.

In [15]:
print('Y =\n', Y.data, '\n\nY.subset(col=[-2]) = \n', Y.subset(col=[-2]).data)

Y =
 [[1.  0.  0. ]
 [1.  0.  0. ]
 [0.  2.  0. ]
 [0.  1.  2.5]
 [0.  0.  2. ]
 [0.  0.  1. ]] 

Y.subset(col=[-2]) = 
 [[1.  0. ]
 [1.  0. ]
 [0.  0. ]
 [0.  2.5]
 [0.  2. ]
 [0.  1. ]]


### `diagstack(by='row')`

Diagonalizes each row (default) or column of a matrix and stacks them vertically or horizontally.

In [16]:
print('Y =\n', Y.data, '\n\nY.diagstack() = \n', Y.diagstack().data)

Y =
 [[1.  0.  0. ]
 [1.  0.  0. ]
 [0.  2.  0. ]
 [0.  1.  2.5]
 [0.  0.  2. ]
 [0.  0.  1. ]] 

Y.diagstack() = 
 [[1.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [1.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  2.  0. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.  2.5]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  2. ]
 [0.  0.  0. ]
 [0.  0.  0. ]
 [0.  0.  1. ]]


### Other notes

Methods can be chained. For example, the following sums the Y matrix by rows then diagonalizes it.

In [17]:
print('Y =\n', Y.data, '\n\nY.row_sum().diag() = \n', Y.row_sum().diag().data)

Y =
 [[1.  0.  0. ]
 [1.  0.  0. ]
 [0.  2.  0. ]
 [0.  1.  2.5]
 [0.  0.  2. ]
 [0.  0.  1. ]] 

Y.row_sum().diag() = 
 [[1.  0.  0.  0.  0.  0. ]
 [0.  1.  0.  0.  0.  0. ]
 [0.  0.  2.  0.  0.  0. ]
 [0.  0.  0.  3.5 0.  0. ]
 [0.  0.  0.  0.  2.  0. ]
 [0.  0.  0.  0.  0.  1. ]]
